In [1]:
# !pip install tensorflow==2.12
# !pip install tensorflow_io==0.23.1

In [2]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import tensorflow as tf
from PIL import Image
import numpy as np
from MongoDBDataset import create_dataset
import datetime

load_dotenv()

True

In [3]:
client = MongoClient(os.environ["URI"])
db = client.flowers
N_CLASSES = len(db.test.distinct("category"))

In [4]:
train_dataset = create_dataset(db.train, batch_size=3, mode="train")
eval_dataset = create_dataset(db.test, batch_size=1, mode="test")

In [5]:
import keras.layers as layers


def resnet_block(inputs, n_channels=40):
    skip = layers.Conv2D(filters=n_channels, kernel_size=1, padding="same")(inputs)
    skip = layers.BatchNormalization()(skip)
    for n in range(3):
        inputs = layers.Conv2D(
            filters=n_channels,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding="same",
            name=f"resblock_{n_channels}_{n}",
        )(inputs)
        inputs = layers.BatchNormalization()(inputs)
        inputs = layers.LeakyReLU()(inputs)

    inputs = inputs + skip
    inputs = layers.LeakyReLU()(inputs)
    inputs = layers.MaxPool2D()(inputs)
    return inputs


def create_resnet():
    input = layers.Input(name="flower_image", shape=(256, 256, 3), dtype=tf.float32)
    output = input
    blocks = [64, 128, 256]  # 512, 1048
    for i in blocks:
        output = resnet_block(output, i)
    output = layers.GlobalAveragePooling2D()(output)
    output = layers.Dense(N_CLASSES, activation="softmax", name="layers_classifier")(output)
    return tf.keras.models.Model(inputs=input, outputs=output, name=f"Resnet_{len(blocks)}")


def create_model():
    input = layers.Input(name="flower_image", shape=(256, 256, 3), dtype=tf.float32)

    model_design = [
        layers.Rescaling(1 / 127.5, -1),
        # layers.Lambda(tf.image.resize_with_crop_or_pad, arguments=(256, 256)),
        layers.Conv2D(
            filters=128,
            kernel_size=(5, 5),
            strides=(2, 2),
            activation="relu",
            name="layers_conv2d_1",
        ),
        layers.MaxPooling2D(),  # 256 -> 128
        layers.Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=(1, 1),
            activation="relu",
            name="layers_conv2d_2",
        ),
        layers.Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=(1, 1),
            activation="relu",
            name="layers_conv2d_3",
        ),
        layers.Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=(1, 1),
            activation="relu",
            name="layers_conv2d_4",
        ),
        layers.MaxPooling2D(),  # 128 -> 64
        layers.Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=(1, 1),
            activation="relu",
            name="layers_conv2d_5",
        ),
        layers.MaxPooling2D(),  # 64 -> 32
        layers.Flatten(name="layers_flatten"),  # 32*32*64 = 65536
        layers.Dropout(0.2, name="layers_dropout"),
        layers.Dense(N_CLASSES, activation="softmax", name="layers_classifier"),
    ]
    output = input

    for layer in model_design:
        output = layer(output)

    return tf.keras.models.Model(inputs=input, outputs=output, name="FlowerFinder")

In [6]:
model = create_resnet()
model.summary()

Model: "Resnet_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 flower_image (InputLayer)   [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 resblock_64_0 (Conv2D)      (None, 256, 256, 64)         1792      ['flower_image[0][0]']        
                                                                                                  
 batch_normalization_1 (Bat  (None, 256, 256, 64)         256       ['resblock_64_0[0][0]']       
 chNormalization)                                                                                 
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 256, 256, 64)         0         ['batch_normalization_1

In [7]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[
        "accuracy",
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
    ],
)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1, update_freq="batch"
)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("models", save_best_only=True)

model.fit(train_dataset, epochs=10, callbacks=[tensorboard_callback], validation_data=eval_dataset)

Epoch 1/10
    370/Unknown - 1901s 5s/step - loss: 2.3263 - accuracy: 0.2297 - precision: 0.3697 - recall: 0.0396

KeyboardInterrupt: 

In [ ]:
client.close()